<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/Multileyer_Perceptron_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multilayer-Perceptron con tensorflow y keras v3 (Dividiendo los datos de entrenamiento en lotes)
En este nuevo notebook vamos a crear una arquitectura de nua red neuronal con TF y keras es una version actual del algoritmo descrito en [multilayer v1](https://github.com/TottiPuc/Machine_learning/blob/master/Multileyer_Perceptron_v1.ipynb). donde para un grande volumen de datos se requiere dividir el dataset en lotes o *batches* para liberar memoria de nuestra maquina

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import tensorflow as tf

Creamos los hiperparametros de la red y los datos sinteticos a ser entrenados

In [0]:
size = 200000
num_empochs = 10
learning_rate = 0.001
n_batches = 10000
batch_size = 100
#creamos 200000 datos aleatorios enteros entre 0 y 100
x1 = np.random.randint(0,100, size)
x2 = np.random.randint(0,100, size)
x_train = np.dstack((x1,x2))[0] # concatenamos x1 y x2 y tomamos la primera capa de profundidad

Creamos una funcion Y que dependa de x para generas nuestros datos historicos

In [4]:
y_train = 3*(x1**(1/2))+2*(x2**2)
print( "estos son los valores de entrenamiento para 'x' y 'y' y sus respectivas dimensiones ")
print("valores de x {}, con dimension {}".format(x_train,x_train.shape))
print("valores de y {}, con dimension {}".format(y_train,y_train.shape))

estos son los valores de entrenamiento para 'x' y 'y' y sus respectivas dimensiones 
valores de x [[40 40]
 [38 67]
 [14 75]
 ...
 [67  1]
 [44 15]
 [85 96]], con dimension (200000, 2)
valores de y [ 3218.97366596  8996.49324201 11261.22497216 ...    26.55605832
   469.89974874 18459.65863337], con dimension (200000,)


Creamos el modelo con la herramienta keras y su clase sequential que nos permite adicionar capa 
tras capa dependiendo de lo que queramos construir. A diferencia de la V1 aumentaremos una capa oculta para compensar el tiempo de entrenamiento causado por la división en lotes del conjunto de entrenamiento

In [0]:
modelo = tf.keras.Sequential()
modelo.add(tf.keras.layers.Dense(64, input_shape=(2,), activation='sigmoid'))
modelo.add(tf.keras.layers.Dense(128, activation='relu'))
modelo.add(tf.keras.layers.Dense(128, activation='relu'))
modelo.add(tf.keras.layers.Dense(2))

# creamos el optimizador
optimizador = tf.keras.optimizers.Adam(learning_rate)

#reamos la funcion de custo
mse = tf.keras.losses.MSE

Compilamos nuestra arquitectura

In [0]:
modelo.compile(optimizador,mse)

La nueva variante de este modelo es la creación de lotes en la etapa de entrenamiento para poder liberar memoria en el caso de tener una cantidad grande de datos. Para eso dividiremos nuestros datos en pequeños lotes

In [0]:
dataset_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
dataset_train = dataset_train.repeat().shuffle(x_train.shape[0]).batch(batch_size)

Para crear la funcion de entrenamiento debemos usar un loop donde iremos almacenando los calculos de los gradientes en cada lote, es este punto el que facilita el uso de esta técnica en modo online.

In [18]:
print("Iniciando el entrenamiento del modelo por favor espere ...")
for i, (batch_xs_train, batch_ys_train) in enumerate(dataset_train.take(n_batches),1):
  
  #calculamos la predicción y la tasa de error
  with tf.GradientTape() as tape:
    #prediccion del modelo
    y_pred = modelo(batch_xs_train)

    #error del modelo
    loss = mse(batch_ys_train[0], y_pred[0])

    #calculo el gradiente
    grads = tape.gradient(loss, modelo.trainable_variables)  # en trainable_variables estan los pesos y bias almacenados en cada pasada

    # separamos los gradientes de forma unica
    gradientes_process = [g for g in grads]

    # obtenemos los gradientes y pesos de cada lote
    grads_y_pesos = zip(gradientes_process,modelo.trainable_variables)

  # optimizamos los pesos con el valor de los gradientes
  optimizador.apply_gradients(grads_y_pesos)

print("Tasa de rror final del modelo en entrenamiento ",np.mean(loss.numpy()))

Iniciando el entrenamiento del modelo por favor espere ...
Tasa de rror final del modelo en entrenamiento  668660.0


In [19]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 25,282
Trainable params: 25,282
Non-trainable params: 0
_________________________________________________________________


#Evaluando el modelo
Generamos nuevos datos sinteticos para evaluar el modelo ( tienen que estar en el mismo rango entre 0 y 100 ya que el entrenamiento fue realizado con este rango de datos) solo que esta vez se usa el mismo concepto de lotes para poder testar el modelo

In [0]:
x3= np.array([100, 9, 62, 79, 94, 91, 71, 41])
x4= np.array([65, 39, 40, 44, 77, 42, 36, 74])
x_test = np.dstack((x3, x4))[0]

y_test = 3*(x3**(1/2)) + 2*(x4**2)

Generamos los lotes de teste

In [0]:
dataset_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
dataset_test = dataset_test.repeat().shuffle(x_test.shape[0]).batch(batch_size)

In [22]:
print("Iniciamos la evaluación del modelo")
for i ,(batch_xs_test, batch_ys_test) in enumerate(dataset_test.take(n_batches),1):
  y_pred = modelo(batch_xs_test)
  loss = mse(batch_ys_test[0], y_pred)
print("Tasa de erro final de la etapa de test ", np.mean(loss.numpy()))

Iniciamos la evaluación del modelo
Tasa de erro final de la etapa de test  10420102.0


Por ultimo hacemos algunas predicciones con nuestro modelo

In [25]:
y_pred = modelo.predict(x_test)

print("\n")

for i in range(5):
    print ('''Entrada(x): ({}, {}), Salida(y): ({:.0f}), Predicción del Modelo(y_pred): ({:.0f})'''.format(x1[i], x2[i], y_test[i], y_pred[i][0]))

print("\n")



Entrada(x): (42, 27), Salida(y): (8480), Predicción del Modelo(y_pred): (7161)
Entrada(x): (60, 13), Salida(y): (3051), Predicción del Modelo(y_pred): (7146)
Entrada(x): (0, 17), Salida(y): (3224), Predicción del Modelo(y_pred): (7138)
Entrada(x): (98, 81), Salida(y): (3899), Predicción del Modelo(y_pred): (7180)
Entrada(x): (4, 97), Salida(y): (11887), Predicción del Modelo(y_pred): (7020)




#Guardando el modelo

In [0]:
# con esto se salva modelos
modelo.save('modelo-v.3')
#con estose carga modelos
modelo-v3 = tf.keras.models.load_model('modelo-v.3')